In [43]:
CLIENT_SECRETS_FILE = "client_secret.json"

In [44]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [45]:
import google.oauth2.credentials
import os
 
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
 
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [46]:
if __name__ == '__main__':
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()
 

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=500298203057-ocg8hmq5q31f9cakrm5svokga1m651vn.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=N6ny2uDRWVUBIAKMMDycsmjtKzUGxu&prompt=consent&access_type=offline
Enter the authorization code: 4/1gE1J-K9mCoMLbRBUOG3ipWBR4Ijp_6s23yWPbhJBql5PEHMSniWDmI


In [47]:
def recherche_videos_par_mot(service, **kwargs):
    results = service.search().list(**kwargs).execute()
    for item in results['items']:
        print('Titre : %s - Video id : %s' % (item['snippet']['title'], item['id']['videoId']))
 

keyword = input('Enter a keyword: ')
recherche_videos_par_mot(service, q=keyword, part='id,snippet', eventType='completed', type='video')
 

Enter a keyword: python


HttpError: <HttpError 403 when requesting https://www.googleapis.com/youtube/v3/search?q=python&part=id%2Csnippet&eventType=completed&type=video&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.">

In [84]:
def get_video_comments(service, **kwargs):
    comments = []
    results = service.commentThreads().list(**kwargs).execute()
 
    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)
 
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break
 
    return comments

In [51]:
def recherche_videos_par_mot_2(service, **kwargs):
    results = get_videos(service, **kwargs)
    final_result = []
    for item in results:
        title = item['snippet']['title']
        video_id = item['id']['videoId']
        comments = get_video_comments(service, part='snippet', videoId=video_id, textFormat='plainText')
        final_result.extend([(video_id, title, comment) for comment in comments])

In [83]:

def get_videos(service, **kwargs):
    final_results = []
    results = service.search().list(**kwargs).execute()
 
    i = 0
    max_pages = 3
    while results and i < max_pages:
        final_results.extend(results['items'])
        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.search().list(**kwargs).execute()
            i += 1
        else:
            break
 
    return final_results
 

In [91]:
keyword = input('Enter a keyword: ')
search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')

Enter a keyword: python
['Audible sir', 'Know  only  atten this class']
['สอนเรื่องยาก เป็นเรื่องง่ายเลยครับ สุดยอดครับอาจารย์', 'Slow is sleep  ดูหลายครั้งเลยกว่าจะจบ555555ขอบคุณมากครับ', 'ชอบมากครับเข้าใจง่ายมาก  เทียบกับผมเรียนในห้อง ไม่เข้าใจเท่ากับลุงสอนเลย อยากให้ทำต่อไปครับ <3', 'ขอบคุณครับ', 'ขอถามหน่อยครับส่วนใหญ่แล้วโปรแกรมเมอร์ ทำอะไรบ้างครับ เช่น เขียนโค้ดเกม', 'ชอบลุงสอนมากครับผมเริ่มจาก 0', 'ขอบคุณครับ', 'ขอบคุณที่ให้คนแก่เรียนแค่ประถมต้นได้เรียน', 'สุดยอด\u200bๆ\u200bๆ\u200bครับ\u200b อาจารย์\u200b', 'ถ้าขึ้น Traceback ตอนใส่คำสั่ง ลองดุที่เราพิมคำสั่งดีๆครับ ของผมขึ้น Traceback เพราะ พิมคำว่า moveto แต่ต้องพิม moveTo ครับ T ต้องตัวใหญ่ โปรแกรมเลยรันปกติ', 'ต้องการเป็นเฮกเกอร์', 'เรียนภาษาที่ไม่ใช่ภาษามนุษย์ 👍👍👍👍', 'เราจำเป็นที่จะต้องไปเรียนไหมการเขียนโปรแกรมที่เรียนเสียเงินและถ้าเรัยนเสียเงินจะต่างเรียนไม่เสียเงินไหมและถ้าเรียนแล้วต้องการพัฒนาการเขียนโปรแกรมให้ต่อเนี่องจนเก่งควรหาสนามฝึกการเขียนโปรแกรมได้ที่ไหนบ้างขอบคุณสำหรับคำแนะนำครับ', 'เราเรียนสายคอมที่เทคนิคเข้านิ

['visual studio code гораздо проще и с таки миже возможностями', 'А что делать если я скачал PyCharm всё нормально... И... Он не запускается я через процесс хакер проверил что происходит и когда я его запускаю , он запускается но через 5 секунд выключается что происходит... ?', 'Здравствуйте можно пожалуйста узнать, можно ли закачать пайтон на слабом ноутбуке', 'также удобно использовать vs code, рекомендую', 'Здравствуйте.\nВыдаёт ошибку при импортировании файла "background.png".\nПишет, мол такой директории не существует.\nКонтраст кода не помог.\nВ чём ошибка?\n\nМеняю window.bgpic() на window.bgcolor() и всё работает.\nМожет не так распаковал?', '2', 'А что прогеры на Python гуглят в Яндексе?', 'Дебуг', 'Я чайник', 'Установил Python', '1', 'И кому будет нужна ваша игра сляпанная на питоне?', 'Здравствуйте. Скиньте пожалуйста ссылку на картинки для этой игры', 'а как он включил? То есть, я код то переписал,но не пойму как ее запустить', 'люди пж скиньте код программы, так как я где 

['Poli poli']
['Got a question on the topic? Please share it in the comment section below and our experts will answer it for you. For Edureka Python Course curriculum, Visit our Website: http://bit.ly/2OpzQWw', 'Very helpful vedio now i really have clear picture on this topic!\nThanks a lot :D', 'Nice I knew it all already', 'you are amazing!!!', 'very nice video', 'which option suits best for python function? 1. function is used to do specific task 2. function is used to create an object 3 . function is used to run the code faster', 'Thanks! I touched on this concept in CodeAcademy and needed more explanation. thanks!', 'awesome', 'Nice Explanation. Very helpful , Thank you!!', 'Python does not support abstract class and Interfaces. Am I correct?', "Hi \n\nPlease send me Python videos in order to understand\n\nI'm not getting the order of videos to learn python\n\nPlease send me link to understand python\n\nTq\nCharan", 'Need detail videos on Abstraction and Encapsulation', 'Great wor

In [49]:
def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    for item in results:
        title = item['snippet']['title']
        video_id = item['id']['videoId']
        comments = get_video_comments(service, part='snippet', videoId=video_id, textFormat='plainText')
        
        print(comments)
        ecrire_csv(comments)

In [48]:
import csv
 
def ecrire_csv(comments):
    with open('comment.csv', 'w') as comments_file:
        comments_writer = csv.writer(comments_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        comments_writer.writerow(['Video ID', 'Title', 'Comment'])
        for row in comments:
            comments_writer.writerow(list(row))
 

In [50]:
def import_commentaires(youtube, video_id):
  results = youtube.commentThreads().list(
    part="snippet",
    maxResults=100,
    videoId=video_id,
    textFormat="plainText"
  ).execute()

  for item in results["items"]:
    comment = item["snippet"]["topLevelComment"]
    author = comment["snippet"]["authorDisplayName"]
    text = comment["snippet"]["textDisplay"]
    print("Commentaire par %s: %s" % (author, text))

  return results["items"]

import_commentaires(service, "rfscVS0vtbw")

Commentaire par freeCodeCamp.org: Click the "JOIN" button below the video to support freeCodeCamp.org!
Commentaire par Age of destruction: Personal reminder 35:20
Commentaire par Ashesh Tamang: This lockdown made me learn Python.
Commentaire par Life of Pyth: Thank you for giving free classes. No ads, too. I salute you, man. You're changing lives. :D
Commentaire par BE_A_57: Difference btwn append and extend() ???
Commentaire par Blaise Ager: Ich war misstrauisch, weil mein Mann mich betrogen, also beschloss ich, seine WhatsApp zu hacken, um die Wahrheit herauszufinden.  Ich benutzte einen Hacker namens CYBERSECURITY CORPORATION, der mir half, das Telefon meines Mannes innerhalb von sechs Stunden zu hacken, und ich fand die Wahrheit, dass mein Mann betrügte.  Sie können den Hack erreichen, indem Sie cybersecteam1@gmailcom oder WhatsApp unter +1(872)3101266 per E-Mail senden
Commentaire par Roj Atanque: Got nothing to do during quarantine so I decided to learn Python and the Khmer Langu

[{'etag': 'xpiDnJiLx4nygC9Io0yf6mOPVZM',
  'id': 'Ugzk263UWvcZS5EsSqp4AaABAg',
  'kind': 'youtube#commentThread',
  'snippet': {'canReply': True,
   'isPublic': True,
   'topLevelComment': {'etag': 'VZIjt8OMdvmL3Bq8488vK_Wmvw0',
    'id': 'Ugzk263UWvcZS5EsSqp4AaABAg',
    'kind': 'youtube#comment',
    'snippet': {'authorChannelId': {'value': 'UC8butISFwT-Wl7EV0hUK0BQ'},
     'authorChannelUrl': 'http://www.youtube.com/channel/UC8butISFwT-Wl7EV0hUK0BQ',
     'authorDisplayName': 'freeCodeCamp.org',
     'authorProfileImageUrl': 'https://yt3.ggpht.com/a/AATXAJwFt03RAznOsPwlfo5c1kW1rp-1o3Xgpw9MNreQMQ=s48-c-k-c0xffffffff-no-rj-mo',
     'canRate': True,
     'likeCount': 846,
     'publishedAt': '2020-04-08T23:07:35Z',
     'textDisplay': 'Click the "JOIN" button below the video to support freeCodeCamp.org!',
     'textOriginal': 'Click the "JOIN" button below the video to support freeCodeCamp.org!',
     'updatedAt': '2020-04-08T23:07:35Z',
     'videoId': 'rfscVS0vtbw',
     'viewerRati